# Chapter 3: Simulating Impacts of Rainfall Variability on Household Food Security

 [⬅️ Previous Chapter](Chapter 2.ipynb)     [➡️ Next Chapter](Chapter 4.ipynb)


## Chapter Objectives

* List
* Of 
* Things
* We
* Will
* Learn